## Selekcja cech

W ramach ćwiczenia będą stosowane kryteria oceny cech opisujących dokumenty ze zbioru `fetch20newsgroups` z punktu widzenia zadania klasyfikacji. Do eksperymentów wybierz dokumenty z kilku grup dyskusyjnych, które będą stanowiły poszczególne klasy (wybrane kategorie należy odkomentować).

In [ ]:
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_classif
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

categories = [
    #'alt.atheism',
    'comp.graphics',
    #'comp.os.ms-windows.misc',
    #'comp.sys.ibm.pc.hardware',
    #'comp.sys.mac.hardware',
    #'comp.windows.x',
    #'misc.forsale',
    'rec.autos',
    #'rec.motorcycles',
    #'rec.sport.baseball',
    #'rec.sport.hockey',
    #'sci.crypt',
    #'sci.electronics',
    #'sci.med',
    'sci.space',
    #'soc.religion.christian',
    'talk.politics.guns',
    #'talk.politics.mideast',
    #'talk.politics.misc',
    #'talk.religion.misc'
]
dataset = fetch_20newsgroups(subset='all', categories = categories, shuffle=True, random_state=42)

In [ ]:
print("Liczba zaladowanych dokumentow wynosi ", len(dataset.data))
print("Naleza one do jednej z ", len(dataset.target_names), " kategorii.")

Przykładowa wiadomość z wczytanego zbioru wygląda następująco:

In [ ]:
doc_number = 5
print("Przykladowa wiadomosc z grupy ", dataset.target_names[dataset.target[doc_number]])
dataset.data[doc_number]

Każdy dokument będzie reprezentowany przez wektor cech, którego składowe są wagami dla słów branych pod uwagę w analizowanym zbiorze dokumentów. Do wyznaczenia tych wektorów cech służy klasa `TfidfVectorizer`. 

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

Liczba cech opisujących dokumenty może być równa, w skrajnym przypadku, liczbie wszystkich słów występujących we wszystkich dokumentach w zbiorze.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, max_features = None, stop_words='english')
docs_vectors = vectorizer.fit_transform(dataset.data)
print("Liczba dokumentów: ", docs_vectors.shape[0])
print("Liczba cech: ", docs_vectors.shape[1])


Dla danego dokumentu składowa i-ta tego wektora cech to liczba wystąpień i-tego słowa (cechy) w tym dokumencie pomnożona przed  współczynni idf, który zależy od całkowitej liczby dokumentów w zbiorze oraz liczby dokumentów zawierającyh słowo i-te. Szczegółowy opis wyznaczania tych cech można znaleźć w dokumentacji 
http://scikit-learn.org/stable/modules/feature_extraction.html
Wektor wartości idf dla wszystkich słów (cech) można odczytać w następujący sposób:

In [ ]:
vectorizer.idf_

Metoda `get_feature_names_out` pozwala na odczytanie nazw wszystkich cech, czyli słów branych pod uwagę podczas analizowania zbioru dokumentów. Są one posortowane alfabetycznie. 

In [ ]:
terms = vectorizer.get_feature_names_out()
terms[-20:-1]

Podział zbioru na dane uczące i testowe.

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test, target_train, target_test = train_test_split(docs_vectors, dataset.target, test_size=0.33, random_state=0)

Uczenie klasyfikatora.

In [ ]:
from sklearn import tree
tree = tree.DecisionTreeClassifier()
tree.fit(data_train, target_train)
print("Blad klasyfikacji dla danych testowych wynosi: ", 1.0 - tree.score(data_test,target_test))

Do selekcji cech można wykorzystać klasę `SelectKBest` pozwalającą na ocenę cech na podstawie wybranej miary podanej jako parametr. 

http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest

Po wywołaniu metody `fit` zostaną obliczone wartości miary dla wszystkich cech. Można je odczytać z atrybutu `scores_`.

Metoda `transform` przekształca zbiór danych zostawiając tylko zadaną liczbę cech o najwyższych wartościach.

Metoda `fit_transform` ocenia cechy a następnie przekształca zbiór danych (efekt jak po zastosowaniu kolejno `fit` oraz `transform`).

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
select_mutual = SelectKBest(chi2,k=20)
select_mutual.fit(data_train, target_train)
select_mutual.scores_

**Zadanie 1 (2 pkt.)**

Zmodyfikuj zbiór danych zostawiając N cech o największych wartościach miary chi2. Przeprowadź uczenie i testowanie drzewa po tej modyfikacji. Czy redukcja liczby cech wpłynęła na zmianę błędu klasyfikacji? Przeprowadź próby przynajmniej dla 5 różnych wartości N.

**Zadanie 2 (1 pkt.)**

Wyświetl 30 słów odpowiadających cechom o największych wartościach miary chi2. Czy widać ich związek z wybranymi klasami (kategoriami)?

## Ekstrakcja cech

Jedną z podstawowych metod ekstrakcji cech jest analiza głównych składowych (ang. principal component analysis - PCA) zaimplementowana w klasie `PCA`. Podstawowym parametrem metody jest parametr `n_components` decydujcy o liczbie cech po transformacji.

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=10)
X = digits.data
y = digits.target
n_samples, n_features = X.shape
pca = PCA(n_components = 2)
X_new = pca.fit_transform(X)
plt.scatter(X_new[:,0], X_new[:,1],c=y)
plt.show()

Inną metodą ekstrakcji jest liniowa analiza dyskryminacyjna, która w przeciwieństwie do PCA uwzględnia informację o przynależności danych do poszczególnych klas.

http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=2)
X_lda = lda.fit_transform(X, y)
plt.scatter(X_lda[:,0], X_lda[:,1], c=y)
plt.show()

**Zadanie 3 (1,5 pkt.)**

Dla danych `digits` przeprowadź analizę głównych składowych i narysuj tzw. wykres osypiska, czyli wykres pokazujący wartości wariancji dla kolejnych składowych głównych. Wykres powinien być narysowany dla wszystkich składowych. Wartości wariancji można odczytać z atrybutu `pca.explained_variance_ratio_`.

**Zadanie 4 (1,5 pkt.)**

Napisz funkcję, która zwraca liczbę cech, jakie należy uwzględnić, aby zachować po przeprowadzeniu analizy głównych składowych część wariancji (suma wariancji związanych z poszczególnymi składowymi) podaną jako parametr funkcji. Parametr ten może przyjmować wartości z przedziału (0;1]. Jesli jako parametr wejściowy podano wartość 1, co oznacza 100% wariancji, funkcja powinna zwrócić maksymalną liczbą cech.

**Zadanie 5 (1,5 pkt)**

Zredukuj liczbę cech wybranego zbioru danych stosując analizę głównych składowych. O ostatecznej liczbie cech zdecyduj na podstawie ilości zachowanej wariancji. Przeprowadź uczenie i testowanie wybranego klasyfikatora na tych danych.

Uwaga 1: liczba cech po transformacji będzie równa wartości `n_components` lub liczbie przykładów w zbiorze jeśli jest ich mniej niż `n_components`.

Uwaga 2: analiza głownych składowych powinna być przeprowadzona na podstawie danych uczących, natomiast dane testowe powinny być przetworzone na podstawie macierzy transformacji otrzymanej w wyniku przeprowadzenia PCA na danych uczących. Odpowiedz na pytanie, dlaczego zastosowanie PCA na pełnym zbiorze danych (uczące + testowe) jest niewłaściwe.

**Zadanie 6 (1 pkt.)**

Wykonaj analogiczne zadanie stosując wybraną metodę selekcji cech zamiast analizy głównych składowych. Porówaj wyniki zwracając uwagę na to, ile cech należy wybrać, aby otrzymać wynik na poziomie takim, jak otrzymano w zadaniu 6.

# Dobór optymalnych wartości parametrów

Klasa `GridSearchCV` umożliwia przeprowadzenie uczenia modelu i testowania dla różnych parametrów i wybór zestawu parametrów optymalnych z punktu widzenia danego kryterium (np. w przypadku klasyfikatorów tym kryterium jest minimalny błąd klasyfikacji). Testowanie modelu dla różnych wartości parametrów odbywa się w procesie walidacji krzyżowej.

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

W poniższym przykładzie uczone będzie drzewo decyzyjne (dla danych `digits`) dla różnych kryteriów wyboró atrybutu oraz różnych wartości minimalnej liczby przykładów w liściach.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix #, accuracy_score
data_train, data_test, target_train, target_test = train_test_split(digits.data, digits.target, test_size=0.3, random_state=0)
tree = DecisionTreeClassifier()
parameters ={'criterion': ['gini', 'entropy'], 'min_samples_leaf': [5,4,3]}
search = GridSearchCV(tree, parameters, cv=5)
search.fit(data_train, target_train)

Wartości parametrów, dla których osiągnięto najlepszy wynik są zapisane w atrybucie `best_params_`.

In [ ]:
search.best_params_

Przetestowanie modelu dla najlepszego zestawu parametrów.

In [ ]:
search.score(data_test, target_test)

# Przetwarzanie potokowe

Jeżeli nasz proces składa się z kilku etapów można wykorzystać klasę `Pipeline`.

http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

W poniższym przykładzie zdefiniowano proces składający się z ekstaraktora cech (PCA) oraz klasyfikatora (drzewo decyzyjne).

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('extract', PCA()), ('classify', DecisionTreeClassifier())])

Przetwarzanie potokowe zwykle łączy się z poszukiwaniem optymalnego zestawu parametrów. Jako parametr funkcji `GridSearchCV` należy wtedy podać obiekt reprezentujący nasz wieloetapowy proces. W przykładzie poniżej dla zdefiniowanego potoku odbywa się poszukiwanie optymalnej wartości liczby składowych dl ametody PCA. Nazwy prametrów tworzy się łącząc nadaną przez nas nazwę etapu (`extract`) z nazwą odpowiedniego parametru (`n_components`) za pomocą znaków podkreślenia.

In [ ]:
parameters = {'extract__n_components': [10,20,30]}
search = GridSearchCV(pipe, parameters, cv=5)
search.fit(data_train, target_train)

In [ ]:
search.best_params_

In [ ]:
search.score(data_test, target_test)

**Zadanie 7 (1,5 pkt.)**

Dla wybranego zbioru danych zaprojektuj model składający się z etapu selekcji cech, ekstrakcji cech i klasyfikacji. Optymalne parametry selekcji i ekstrakcji dobierz korzystając z klasy `GridSearchCV`. Możesz wybrać dowolny klasyfikator. 